http://ithelp.ithome.com.tw/articles/10187569

# 前言

## 隨機森林分類器
屬於建構於決策樹之上的整體學習應用，

每一個基本分類器都是一個決策樹。這時我們心中就冒出一個疑問：

隨機森林跟以決策樹為基本分類器構成的 Bagging 有什麼不同？

>最大的差異應該就是隨機的部分，以決策樹為基本分類器構成的 Bagging 的 Boostrap sampling 只有應用在列方向（觀測值方向）；
>隨機森林的 bootstrap sampling 則是同時應用在列方向（觀測值方向）與欄方向（變數方向）。

## 支持向量機
則是一種利用最適化（Optimization）概念在模型的精確度以及推廣能力（Generalization ability）中取得一個最佳平衡點的演算法，
她在面對小樣本，非線性與多維度的資料中廣受歡迎。

# 隨機森林

隨機森林演算法會對資料從列方向（觀測值方向）與欄方向（變數方向）進行 Bootstrap sampling，

得到不同的訓練資料，然後根據這些訓練資料得到一系列的決策樹分類器，假如產生了 5 個決策樹分類器，

她們對某個觀測值的預測結果分別為 1, 0, 1, 1, 1，那麼隨機森林演算法的輸出結果就會是 1，

這個過程與 Bagging 演算法相同，同樣稱為基本分類器的投票。

隨機森林演算法在面對變數具有多元共線性或者不平衡資料（Unbalanced data）的情況時是倍受青睞的演算法。

我們使用 sklearn.ensemble 的 RandomForestClassifier()。

In [1]:
import numpy as np
import pandas as pd
from sklearn import cross_validation, ensemble, preprocessing, metrics

# 載入資料
url = "https://storage.googleapis.com/2017_ithome_ironman/data/kaggle_titanic_train.csv"
titanic_train = pd.read_csv(url)

# 填補遺漏值
age_median = np.nanmedian(titanic_train["Age"])
new_Age = np.where(titanic_train["Age"].isnull(), age_median, titanic_train["Age"])
titanic_train["Age"] = new_Age

# 創造 dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_Sex = label_encoder.fit_transform(titanic_train["Sex"])

# 建立訓練與測試資料
titanic_X = pd.DataFrame([titanic_train["Pclass"],
                         encoded_Sex,
                         titanic_train["Age"]
]).T
titanic_y = titanic_train["Survived"]
train_X, test_X, train_y, test_y = cross_validation.train_test_split(titanic_X, titanic_y, test_size = 0.3)

# 建立 random forest 模型
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_X, train_y)

# 預測
test_y_predicted = forest.predict(test_X)

# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print(accuracy)

C:\Users\po390\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.809701492537


# 支持向量機

支持向量機是一種最小化結構風險（Structural risk）的演算法，

何謂結構型風險？機器學習的內涵在於假設一個類似模型去逼近真實模型，

而量化類似模型與真實模型之間差距的方式，跟我們在計算績效（準確率）用的概念是相同的，

我們用類似模型預測的結果去跟答案比較。

許多的分類器可以在訓練資料上達到很高的正確率（稱作 Overfitting），

但是卻失去應用在實際問題的推廣能力（Generalization ability）。

資料科學家將分類器在訓練樣本可能過度配適的風險稱為 Empirical risk，

分類器的推廣能力不足的風險稱為 Generalization risk，兩者的總和即為結構風險，

而支持向量機就是在兩者之間取得最佳平衡點，進而得到一個在訓練資料績效不錯，亦能推廣適用的類似模型。

我們使用 sklearn.svm 的 SVC()。

In [2]:
import numpy as np
import pandas as pd
from sklearn import cross_validation, svm, preprocessing, metrics

# 載入資料
url = "https://storage.googleapis.com/2017_ithome_ironman/data/kaggle_titanic_train.csv"
titanic_train = pd.read_csv(url)

# 填補遺漏值
age_median = np.nanmedian(titanic_train["Age"])
new_Age = np.where(titanic_train["Age"].isnull(), age_median, titanic_train["Age"])
titanic_train["Age"] = new_Age

# 創造 dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_Sex = label_encoder.fit_transform(titanic_train["Sex"])

# 建立訓練與測試資料
titanic_X = pd.DataFrame([titanic_train["Pclass"],
                         encoded_Sex,
                         titanic_train["Age"]
]).T
titanic_y = titanic_train["Survived"]
train_X, test_X, train_y, test_y = cross_validation.train_test_split(titanic_X, titanic_y, test_size = 0.3)

# 建立 SVC 模型
svc = svm.SVC()
svc_fit = svc.fit(train_X, train_y)

# 預測
test_y_predicted = svc.predict(test_X)

# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print(accuracy)

0.798507462687


# AUC

在二元分類模型裡面，除了我們一直使用的準確率（Accuracy）以外，

其實還有許多由 Confusion matrix 所衍生的績效指標，像是精確度（Precision）或者召回率（Recall）等，

其中 AUC 是一個常見指標，它同時考慮假警報率（False alarm rate）與命中率（True positive rate），

AUC 愈接近 1，就表示分類效果愈好；愈接近 0.5 就表示分類效果愈差不好。


# 效果	AUC 區間

傑出	AUC 介於 0.9-1.0 之間

優秀	AUC 介於 0.8-0.9 之間

普通	AUC 介於 0.7-0.8 之間

不好	AUC 介於 0.6-0.7 之間

差勁	AUC 介於 0.5-0.6 之間

# 隨機森林分類器的 AUC

在 Python 使用 sklearn.metrics 的 auc()

In [11]:
import numpy as np
import pandas as pd
from sklearn import cross_validation, ensemble, preprocessing, metrics

# 載入資料
url = "https://storage.googleapis.com/2017_ithome_ironman/data/kaggle_titanic_train.csv"
titanic_train = pd.read_csv(url)

# 填補遺漏值
age_median = np.nanmedian(titanic_train["Age"])
new_Age = np.where(titanic_train["Age"].isnull(), age_median, titanic_train["Age"])
titanic_train["Age"] = new_Age

# 創造 dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_Sex = label_encoder.fit_transform(titanic_train["Sex"])

# 建立訓練與測試資料
titanic_X = pd.DataFrame([titanic_train["Pclass"],
                         encoded_Sex,
                         titanic_train["Age"]
]).T
titanic_y = titanic_train["Survived"]
train_X, test_X, train_y, test_y = cross_validation.train_test_split(titanic_X, titanic_y, test_size = 0.3)

# 建立 random forest 模型
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(train_X, train_y)

# 預測
test_y_predicted = forest.predict(test_X)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.75809457256


# 支持向量機分類器的 AUC

In [9]:
import numpy as np
import pandas as pd
from sklearn import cross_validation, svm, preprocessing, metrics

# 載入資料
url = "https://storage.googleapis.com/2017_ithome_ironman/data/kaggle_titanic_train.csv"
titanic_train = pd.read_csv(url)

# 填補遺漏值
age_median = np.nanmedian(titanic_train["Age"])
new_Age = np.where(titanic_train["Age"].isnull(), age_median, titanic_train["Age"])
titanic_train["Age"] = new_Age

# 創造 dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_Sex = label_encoder.fit_transform(titanic_train["Sex"])

# 建立訓練與測試資料
titanic_X = pd.DataFrame([titanic_train["Pclass"],
                         encoded_Sex,
                         titanic_train["Age"]
]).T
titanic_y = titanic_train["Survived"]
train_X, test_X, train_y, test_y = cross_validation.train_test_split(titanic_X, titanic_y, test_size = 0.3)

# 建立 SVC 模型
svc = svm.SVC()
svc_fit = svc.fit(train_X, train_y)

# 預測
test_y_predicted = svc.predict(test_X)

# 績效
fpr, tpr, thresholds = metrics.roc_curve(test_y, test_y_predicted)
auc = metrics.auc(fpr, tpr)
print(auc)

0.76537386443
